In [1]:
import os
import subprocess
import sys
sys.path.append('..')
from tools.formating.formating import *
from tools.utils.utils import *
from tools.qc.scanpy_qc import *
import numpy as np
import scipy

scanpy==1.9.5 anndata==0.9.2 umap==0.5.4 numpy==1.22.4 scipy==1.9.1 pandas==1.3.5 scikit-learn==1.3.1 statsmodels==0.14.0 igraph==0.10.8 louvain==0.8.1 pynndescent==0.5.10


In [2]:
# input = "/ps/ai-ready/data/Clustering/Tabula_Sapiens/TS_germ line.h5ad"
# input = "/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5ad"
# input = "/ps/ai-ready/data/error/hca_heart_neuronal_raw.h5ad"
# input = "/ps/ai-ready/data/Clustering/Tabula_Sapiens/TS_Skin.h5ad"
input = "/ps/ai-ready/data/error/droplet_Bladder_seurat_tiss.h5seurat"
# input = "/ps/ai-ready/data/error/tung.rds"
adata = load_anndata(input)
adata

/ps/ai-ready/data/error/droplet_Bladder_seurat_tiss.h5seurat

    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    Inside convert_seurat_sce_to_anndata
RNA
/ps/ai-ready/data/error/droplet_Bladder_seurat_tiss.h5seurat
[1] "inside R ConvertSeuratSCEtoAnndata"
[1] "Inside LoadSeurat"
[1] "srat object is loaded"
[1] "inside s to a"
[1] "/ps/ai-ready/data/error/droplet_Bladder_seurat_tiss.h5ad"
[1] "AnnData object is saved successfully."
/ps/ai-ready/data/error/droplet_Bladder_seurat_tiss.h5ad
['RNA']


AnnData object with n_obs × n_vars = 2500 × 23341
    obs: 'orig.ident', 'n_counts', 'n_genes', 'channel', 'tissue', 'subtissue', 'mouse.sex', 'mouse.id', 'percent.ercc', 'percent.ribo', 'free_annotation', 'cell_ontology_class', 'res.0.4', 'previous_free_annotation', 'previous_cell_ontology_class', 'cluster.ids', 'cell_ontology_id'
    var: 'name'

In [12]:
adata.obs.head()

,orig.ident,n_counts,n_genes,channel,tissue,subtissue,mouse.sex,mouse.id,percent.ercc,percent.ribo,free_annotation,cell_ontology_class,res.0.4,previous_free_annotation,previous_cell_ontology_class,cluster.ids,cell_ontology_id
10X_P4_3_AAAGTAGAGATGCCAG,10X,10291.0,2853,10X_P4_3,Bladder,,M,3-M-8,0.0,0.219609,Bladder mesenchymal cell,bladder cell,0,NA,NA,0,CL:1001319
10X_P4_3_AACCGCGTCCAACCAA,10X,17520.0,4194,10X_P4_3,Bladder,,M,3-M-8,0.0,0.176712,Bladder mesenchymal cell,bladder cell,2,NA,NA,2,CL:1001319
10X_P4_3_AACTCCCGTCGGGTCT,10X,12944.0,3347,10X_P4_3,Bladder,,M,3-M-8,0.0,0.192444,Bladder mesenchymal cell,bladder cell,5,NA,NA,5,CL:1001319
10X_P4_3_AACTCTTAGTTGCAGG,10X,19594.0,3851,10X_P4_3,Bladder,,M,3-M-8,0.0,0.356385,Luminal bladder epithelial cell,bladder urothelial cell,3,NA,NA,3,CL:1001428
10X_P4_3_AACTCTTTCATAACCG,10X,14594.0,3809,10X_P4_3,Bladder,,M,3-M-8,0.0,0.157119,Bladder mesenchymal cell,bladder cell,2,NA,NA,2,CL:1001319


In [6]:
df = pd.DataFrame.sparse.from_spmatrix(adata.X, index=None, columns=None)
df

,0,1,2,3,4,5,6,7,8,9,...,23331,23332,23333,23334,23335,23336,23337,23338,23339,23340
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,0.0,0.0,0.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2496,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2497,0.0,0.0,0.0,0.0,0.0,3.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2498,0.0,0.0,0.0,4.0,0.0,2.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0


In [ ]:
# Convert Seurat/Single-Cell Experiment object to Anndata object and return the path of Anndata object
def convert_seurat_sce_to_anndata1(path, assay='RNA'):
    import rpy2.rinterface_lib.callbacks as rcb
    import rpy2.robjects as ro
    import anndata2ri
    from rpy2.robjects.packages import importr
    from rpy2.robjects import pandas2ri
    from rpy2.robjects.conversion import localconverter
    
    rcb.logger.setLevel(logging.ERROR)
    ro.pandas2ri.activate()
    anndata2ri.activate()

    # Defining the R script and loading the instance in Python
    ro.r['source'](os.path.abspath("../tools/formating/formating.R"))
    # Access the loaded R functions
    ConvertSeuratSCEtoAnndata_r = ro.globalenv['ConvertSeuratSCEtoAnndata']

    assay_names = None
    adata_path = None
    results = None

    if path.endswith(".h5Seurat") or path.endswith(".h5seurat") or path.endswith(".rds") or path.endswith(".Robj"):
        results = ConvertSeuratSCEtoAnndata_r(path, assay=assay)
        print(results)
        adata_path = str(results.rx2('anndata_path'))
        # adata_path = list(results[2])[0]
        assay_names = list(results[1])
    
    return results

In [ ]:
results = convert_seurat_sce_to_anndata1(input)
results

In [ ]:
type(results)

In [ ]:
from collections import OrderedDict

results = OrderedDict(results)

In [ ]:
results['anndata_path'][0]